In [1]:
# CELL 1: create cleaned_aqi.csv and prominent_pollutant.csv
import os, glob, pandas as pd, calendar, re, numpy as np
DATA_ROOT = r"D:\Downloads\FieldProjectAQI_Data"
DATA_DIR = os.path.join(DATA_ROOT, "data")
IMAGES_DIR = os.path.join(DATA_ROOT, "images")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(IMAGES_DIR, exist_ok=True)
print("DATA_DIR:", DATA_DIR)
print("IMAGES_DIR:", IMAGES_DIR)

# helper to melt calendar layout (Day x Month -> rows)
def melt_calendar_aqi(path, sheet_name='AQI', year_hint=None):
    xls = pd.ExcelFile(path)
    if sheet_name not in xls.sheet_names:
        # fallback: pick first sheet
        sheet = xls.sheet_names[0]
    else:
        sheet = sheet_name
    df = pd.read_excel(path, sheet_name=sheet)
    df.columns = [str(c).strip() for c in df.columns]
    # find Day column
    day_col = next((c for c in df.columns if str(c).strip().lower().startswith('day')), None)
    if day_col is None:
        raise ValueError(f"No Day column in {path} sheet {sheet}")
    months = [c for c in df.columns if c != day_col]
    long = df.melt(id_vars=[day_col], value_vars=months, var_name='MonthName', value_name='AQI')
    long = long.dropna(subset=['AQI']).reset_index(drop=True)
    long[day_col] = pd.to_numeric(long[day_col], errors='coerce').astype('Int64')
    month_map = {calendar.month_name[i]: i for i in range(1,13)}
    month_map.update({calendar.month_abbr[i]: i for i in range(1,13)})
    long['Month'] = long['MonthName'].map(lambda x: month_map.get(str(x).strip(), np.nan)).astype('Int64')
    if year_hint is None:
        m = re.search(r"(20\d{2})", os.path.basename(path))
        year_hint = int(m.group(1)) if m else None
    if year_hint is None:
        raise ValueError("Year not found for file: " + path)
    def mkdate(r):
        try:
            return pd.Timestamp(year=year_hint, month=int(r['Month']), day=int(r[day_col]))
        except:
            return pd.NaT
    long['Date'] = long.apply(mkdate, axis=1)
    long = long.dropna(subset=['Date']).reset_index(drop=True)
    long['AQI'] = pd.to_numeric(long['AQI'], errors='coerce')
    long['Year'] = year_hint
    long = long.rename(columns={day_col:'Day'})
    return long[['Date','Year','Month','Day','AQI']]

# find files and melt only the AQI sheets
xlsx_paths = glob.glob(os.path.join(DATA_DIR, "*.xlsx"))
aqi_parts = []
prom_parts = []
for p in sorted(xlsx_paths):
    xls = pd.ExcelFile(p)
    # If file has 'AQI' sheet, melt AQI
    if any(s.lower()=='aqi' for s in xls.sheet_names):
        try:
            part = melt_calendar_aqi(p, sheet_name='AQI')
            print("AQI melted:", os.path.basename(p), "rows:", len(part))
            aqi_parts.append(part)
        except Exception as e:
            print("AQI melt failed for", p, ":", e)
    # If file has a 'Prominent' or 'Prominent Parameters' sheet, melt pollutant
    poll_sheet = next((s for s in xls.sheet_names if 'prominent' in s.lower() or 'prominent parameter' in s.lower()), None)
    if poll_sheet:
        try:
            # melt pollutant sheet similarly but keep pollutant strings
            dfp = pd.read_excel(p, sheet_name=poll_sheet)
            dfp.columns = [str(c).strip() for c in dfp.columns]
            day_col = next((c for c in dfp.columns if str(c).strip().lower().startswith('day')), None)
            if day_col:
                months = [c for c in dfp.columns if c != day_col]
                longp = dfp.melt(id_vars=[day_col], value_vars=months, var_name='MonthName', value_name='Pollutant')
                longp = longp.dropna(subset=['Pollutant']).reset_index(drop=True)
                longp[day_col] = pd.to_numeric(longp[day_col], errors='coerce').astype('Int64')
                month_map = {calendar.month_name[i]: i for i in range(1,13)}
                month_map.update({calendar.month_abbr[i]: i for i in range(1,13)})
                longp['Month'] = longp['MonthName'].map(lambda x: month_map.get(str(x).strip(), np.nan)).astype('Int64')
                m = re.search(r"(20\d{2})", os.path.basename(p))
                year_hint = int(m.group(1)) if m else None
                def mkdatep(r):
                    try:
                        return pd.Timestamp(year=year_hint, month=int(r['Month']), day=int(r[day_col]))
                    except:
                        return pd.NaT
                longp['Date'] = longp.apply(mkdatep, axis=1)
                longp = longp.dropna(subset=['Date']).reset_index(drop=True)
                longp['Year'] = year_hint
                longp = longp.rename(columns={day_col:'Day'})
                longp = longp[['Date','Year','Month','Day','Pollutant']]
                print("Prominent melted:", os.path.basename(p), "rows:", len(longp))
                prom_parts.append(longp)
        except Exception as e:
            print("Prominent melt failed for", p, ":", e)

if not aqi_parts:
    raise SystemExit("No AQI parts found — check file locations.")
all_long = pd.concat(aqi_parts, ignore_index=True)
print("Combined AQI rows:", len(all_long))

prom_all = pd.concat(prom_parts, ignore_index=True) if prom_parts else pd.DataFrame(columns=['Date','Year','Month','Day','Pollutant'])
print("Combined Prominent pollutant rows:", len(prom_all))

# Save CSVs
cleaned_path = os.path.join(DATA_DIR, "cleaned_aqi.csv")
prom_path = os.path.join(DATA_DIR, "prominent_pollutant.csv")
all_long.to_csv(cleaned_path, index=False)
prom_all.to_csv(prom_path, index=False)
print("Saved cleaned CSV:", cleaned_path)
print("Saved prominent CSV:", prom_path)


DATA_DIR: D:\Downloads\FieldProjectAQI_Data\data
IMAGES_DIR: D:\Downloads\FieldProjectAQI_Data\images
AQI melted: AQI_daily_city_level_mumbai_2023_mumbai_2023.xlsx rows: 365
Prominent melted: AQI_daily_city_level_mumbai_2023_mumbai_2023.xlsx rows: 365
AQI melted: AQI_daily_city_level_mumbai_2024_mumbai_2024.xlsx rows: 366
Prominent melted: AQI_daily_city_level_mumbai_2024_mumbai_2024.xlsx rows: 366
AQI melted: AQI_daily_city_level_mumbai_2025_mumbai_2025.xlsx rows: 90
Prominent melted: AQI_daily_city_level_mumbai_2025_mumbai_2025.xlsx rows: 90
Prominent melted: Prominent_param2023.xlsx rows: 365
Prominent melted: Prominent_param2025.xlsx rows: 90
Prominent melted: prominent_param_2024.xlsx rows: 366
Combined AQI rows: 821
Combined Prominent pollutant rows: 1642
Saved cleaned CSV: D:\Downloads\FieldProjectAQI_Data\data\cleaned_aqi.csv
Saved prominent CSV: D:\Downloads\FieldProjectAQI_Data\data\prominent_pollutant.csv


In [2]:
# CELL 2: create PNG charts in images/ (uses matplotlib)
import os, pandas as pd, numpy as np, matplotlib.pyplot as plt, calendar
DATA_ROOT = r"D:\Downloads\FieldProjectAQI_Data"
DATA_DIR = os.path.join(DATA_ROOT, "data")
IMAGES_DIR = os.path.join(DATA_ROOT, "images")
os.makedirs(IMAGES_DIR, exist_ok=True)

all_long = pd.read_csv(os.path.join(DATA_DIR, "cleaned_aqi.csv"), parse_dates=['Date'])
all_long['Year'] = all_long['Year'].astype(int)
all_long['Month'] = all_long['Month'].astype(int)
print("Plotting from rows:", len(all_long))

# 1) 3-Year Comparative Monthly Averages
monthly_avg = all_long.groupby(['Year','Month'])['AQI'].mean().reset_index()
pivot_monthly = monthly_avg.pivot(index='Month', columns='Year', values='AQI').reindex(index=range(1,13))
plt.figure(figsize=(12,6))
pivot_monthly.plot(kind='bar', width=0.8)
plt.xlabel('Month')
plt.ylabel('Average AQI')
plt.title('3-Year Comparative Monthly Average AQI (Mumbai)')
plt.xticks(ticks=range(12), labels=[calendar.month_abbr[i+1] for i in range(12)], rotation=0)
plt.tight_layout()
p1 = os.path.join(IMAGES_DIR, "3yr_monthly_avg_aqi.png")
plt.savefig(p1, dpi=200)
plt.close()
print("Saved:", p1)

# 2) Overall Annual AQI Trend (line)
annual_avg = all_long.groupby('Year')['AQI'].mean().reset_index()
plt.figure(figsize=(6,4))
plt.plot(annual_avg['Year'], annual_avg['AQI'], marker='o', linewidth=2)
plt.xlabel('Year')
plt.ylabel('Average AQI')
plt.title('Overall Annual Average AQI')
plt.grid(True)
# OPTIONAL: set y-limits dynamically (e.g., pad top)
ymax = max(annual_avg['AQI']) * 1.5
plt.ylim(0, ymax)
p2 = os.path.join(IMAGES_DIR, "annual_avg_aqi.png")
plt.tight_layout()
plt.savefig(p2, dpi=200)
plt.close()
print("Saved:", p2)

# 3) Daily AQI Trends for Each Year
for y in sorted(all_long['Year'].unique()):
    dfy = all_long[all_long['Year']==y].sort_values('Date')
    plt.figure(figsize=(14,4))
    plt.plot(dfy['Date'], dfy['AQI'], linewidth=1)
    plt.xlabel('Date')
    plt.ylabel('AQI')
    plt.title(f'Daily AQI Trend - {y}')
    plt.tight_layout()
    out = os.path.join(IMAGES_DIR, f"daily_trend_{y}.png")
    plt.savefig(out, dpi=150)
    plt.close()
    print("Saved:", out)

# 4) AQI Heatmap (Month vs Day) for each year
import numpy as np
for y in sorted(all_long['Year'].unique()):
    dfy = all_long[all_long['Year']==y].copy()
    heat = dfy.pivot_table(index='Month', columns='Day', values='AQI', aggfunc='mean').reindex(index=range(1,13))
    plt.figure(figsize=(14,6))
    plt.imshow(heat, aspect='auto', origin='lower')
    plt.colorbar(label='AQI')
    plt.yticks(ticks=np.arange(12), labels=[calendar.month_name[i+1] for i in range(12)])
    plt.xticks(ticks=np.arange(31), labels=list(range(1,32)))
    plt.xlabel('Day of Month')
    plt.ylabel('Month')
    plt.title(f'AQI Heatmap (Month vs Day) - {y}')
    plt.tight_layout()
    out = os.path.join(IMAGES_DIR, f"aqi_heatmap_{y}.png")
    plt.savefig(out, dpi=150)
    plt.close()
    print("Saved:", out)

# 5) AQI Category Distribution
def aqi_category(a):
    a = float(a)
    if a <= 50: return 'Good'
    if a <= 100: return 'Satisfactory'
    if a <= 200: return 'Moderate'
    if a <= 300: return 'Poor'
    if a <= 400: return 'Very Poor'
    return 'Severe'

all_long['Category'] = all_long['AQI'].apply(aqi_category)
cat_counts = all_long.groupby(['Year','Category']).size().reset_index(name='days')
pivot_cat = cat_counts.pivot(index='Category', columns='Year', values='days').reindex(index=['Good','Satisfactory','Moderate','Poor','Very Poor','Severe']).fillna(0)
plt.figure(figsize=(8,5))
pivot_cat.plot(kind='bar')
plt.xlabel('AQI Category')
plt.ylabel('Days')
plt.title('AQI Category Distribution by Year')
plt.tight_layout()
out = os.path.join(IMAGES_DIR, "aqi_category_distribution.png")
plt.savefig(out, dpi=150)
plt.close()
print("Saved:", out)

# 6) Monthly AQI Distribution Boxplot
monthly_boxes = [all_long[all_long['Month']==m]['AQI'].dropna().values for m in range(1,13)]
plt.figure(figsize=(12,6))
plt.boxplot(monthly_boxes, labels=[calendar.month_abbr[m] for m in range(1,13)])
plt.xlabel('Month')
plt.ylabel('AQI')
plt.title('Monthly AQI Distribution (Boxplot) - All years combined')
plt.tight_layout()
out = os.path.join(IMAGES_DIR, "monthly_aqi_boxplot.png")
plt.savefig(out, dpi=150)
plt.close()
print("Saved:", out)


Plotting from rows: 821
Saved: D:\Downloads\FieldProjectAQI_Data\images\3yr_monthly_avg_aqi.png
Saved: D:\Downloads\FieldProjectAQI_Data\images\annual_avg_aqi.png
Saved: D:\Downloads\FieldProjectAQI_Data\images\daily_trend_2023.png
Saved: D:\Downloads\FieldProjectAQI_Data\images\daily_trend_2024.png
Saved: D:\Downloads\FieldProjectAQI_Data\images\daily_trend_2025.png
Saved: D:\Downloads\FieldProjectAQI_Data\images\aqi_heatmap_2023.png
Saved: D:\Downloads\FieldProjectAQI_Data\images\aqi_heatmap_2024.png
Saved: D:\Downloads\FieldProjectAQI_Data\images\aqi_heatmap_2025.png
Saved: D:\Downloads\FieldProjectAQI_Data\images\aqi_category_distribution.png


C:\Users\sober\AppData\Local\Temp\ipykernel_12340\84807093.py:105: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(monthly_boxes, labels=[calendar.month_abbr[m] for m in range(1,13)])


Saved: D:\Downloads\FieldProjectAQI_Data\images\monthly_aqi_boxplot.png


<Figure size 1200x600 with 0 Axes>

<Figure size 800x500 with 0 Axes>

In [3]:
# CELL 3: zip images for quick download (optional)
import zipfile, os
IMAGES_DIR = r"D:\Downloads\FieldProjectAQI_Data\images"
zip_path = r"D:\Downloads\FieldProjectAQI_Data\images_backup.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    for root, dirs, files in os.walk(IMAGES_DIR):
        for f in files:
            if f.lower().endswith(".png"):
                zf.write(os.path.join(root, f), arcname=f)
print("Zipped images to:", zip_path)


Zipped images to: D:\Downloads\FieldProjectAQI_Data\images_backup.zip


In [21]:
# RUN THIS CELL: recursive search for .xlsx, preview sheets, and try melting them
import os, glob, pandas as pd, calendar, re, numpy as np
DATA_ROOT = r"D:\Downloads\FieldProjectAQI_Data"   # confirm this is your folder

print("Searching recursively in:", DATA_ROOT)
xlsx_paths = glob.glob(os.path.join(DATA_ROOT, "**", "*.xlsx"), recursive=True)
print(f"Found {len(xlsx_paths)} .xlsx files (paths):")
for p in xlsx_paths:
    print(" -", p)

if not xlsx_paths:
    print("\nNo .xlsx files found under the DATA_ROOT. Please ensure your Excel files (the CPCB downloads) are inside this folder or a subfolder (e.g., 'raw' or 'data/raw').")
else:
    # preview up to 3 files
    preview_count = min(3, len(xlsx_paths))
    previews = []
    for p in xlsx_paths[:preview_count]:
        try:
            xls = pd.ExcelFile(p)
            print(f"\nFile: {p}")
            print("  Sheets:", xls.sheet_names)
            # pick a likely sheet: prefer sheet name containing 'AQI' or first sheet
            sheet = next((s for s in xls.sheet_names if 'aqi' in s.lower()), xls.sheet_names[0])
            print("  Previewing sheet:", sheet)
            df = pd.read_excel(p, sheet_name=sheet, nrows=8)
            print(df.head(8).to_string(index=False))
            previews.append((p, sheet, df.columns.tolist()))
        except Exception as e:
            print("  Could not read file/sheet:", e)

    # If previews exist, try to melt all found files using the calendar-style logic
    if previews:
        def melt_calendar(path, year_hint=None):
            xls = pd.ExcelFile(path)
            # choose sheet containing aqi or first
            sheet = next((s for s in xls.sheet_names if 'aqi' in s.lower()), xls.sheet_names[0])
            df = pd.read_excel(path, sheet_name=sheet)
            df.columns = [str(c).strip() for c in df.columns]
            # find Day column tolerant
            day_col = next((c for c in df.columns if 'day'==str(c).strip().lower() or str(c).strip().lower().startswith('day')), None)
            if day_col is None:
                # try numeric day-like header (1..31) or 'Date'
                if any(str(c).strip().lower()=='date' for c in df.columns):
                    # assume already long format
                    df['Date'] = pd.to_datetime(df[[c for c in df.columns if 'date' in str(c).lower()][0]])
                    if 'AQI' in df.columns:
                        long = df[['Date','AQI']].copy()
                        long['Year'] = long['Date'].dt.year
                        long['Month'] = long['Date'].dt.month
                        long['Day'] = long['Date'].dt.day
                        return long[['Date','Year','Month','Day','AQI']]
                    else:
                        raise ValueError("Found Date column but no AQI column.")
                raise ValueError("No 'Day' column detected and no obvious 'Date' column.")
            # standard calendar melt
            month_cols = [c for c in df.columns if c != day_col]
            long = df.melt(id_vars=[day_col], value_vars=month_cols, var_name='MonthName', value_name='AQI')
            long = long.dropna(subset=['AQI']).reset_index(drop=True)
            long[day_col] = pd.to_numeric(long[day_col], errors='coerce').astype('Int64')
            month_map = {calendar.month_name[i]: i for i in range(1,13)}
            month_map.update({calendar.month_abbr[i]: i for i in range(1,13)})
            long['Month'] = long['MonthName'].map(lambda x: month_map.get(str(x).strip(), np.nan)).astype('Int64')
            # year from filename
            if year_hint is None:
                m = re.search(r"(20\d{2})", os.path.basename(path))
                year_hint = int(m.group(1)) if m else None
            if year_hint is None:
                raise ValueError("Cannot determine year for " + path)
            def make_date(r):
                try:
                    return pd.Timestamp(year=year_hint, month=int(r['Month']), day=int(r[day_col]))
                except:
                    return pd.NaT
            long['Date'] = long.apply(make_date, axis=1)
            long = long.dropna(subset=['Date']).reset_index(drop=True)
            long['AQI'] = pd.to_numeric(long['AQI'], errors='coerce')
            long['Year'] = year_hint
            long = long.rename(columns={day_col:'Day'})
            return long[['Date','Year','Month','Day','AQI']]

        all_parts = []
        for p in xlsx_paths:
            try:
                part = melt_calendar(p)
                print(f"\nMELTED: {p} -> {len(part)} rows")
                all_parts.append(part)
            except Exception as e:
                print(f"\nSKIPPED {p} due to: {e}")

        if all_parts:
            all_long = pd.concat(all_parts, ignore_index=True)
            print("\nCombined all_long rows:", len(all_long))
            print(all_long.head().to_string(index=False))
        else:
            print("\nNo files could be melted automatically. Paste here one small sample of the sheet (first 8 rows and column names) and I will adapt the parser.")


Searching recursively in: D:\Downloads\FieldProjectAQI_Data
Found 6 .xlsx files (paths):
 - D:\Downloads\FieldProjectAQI_Data\data\AQI_daily_city_level_mumbai_2023_mumbai_2023.xlsx
 - D:\Downloads\FieldProjectAQI_Data\data\AQI_daily_city_level_mumbai_2024_mumbai_2024.xlsx
 - D:\Downloads\FieldProjectAQI_Data\data\AQI_daily_city_level_mumbai_2025_mumbai_2025.xlsx
 - D:\Downloads\FieldProjectAQI_Data\data\Prominent_param2023.xlsx
 - D:\Downloads\FieldProjectAQI_Data\data\Prominent_param2025.xlsx
 - D:\Downloads\FieldProjectAQI_Data\data\prominent_param_2024.xlsx

File: D:\Downloads\FieldProjectAQI_Data\data\AQI_daily_city_level_mumbai_2023_mumbai_2023.xlsx
  Sheets: ['AQI', 'Prominent Parameters']
  Previewing sheet: AQI
 Day  January  February  March  April  May  June  July  August  September  October  November  December
   1      147       227    174    108   74    89    67      54         84       87       175       127
   2      188       180    205     93   78    84    54      58   

In [22]:
# CELL B: robustly load calendar-style Excel files and create all_long
import pandas as pd, numpy as np, os, glob, calendar, re
DATA_ROOT = r"D:\Downloads\FieldProjectAQI_Data"   # <-- ensure this matches CELL A
pattern = os.path.join(DATA_ROOT, "*.xlsx")
files = sorted(glob.glob(pattern))
print("Found files:", files)

def melt_calendar(path, year_hint=None):
    xls = pd.ExcelFile(path)
    # choose first sheet or sheet named like 'AQI'
    sheet = None
    for s in xls.sheet_names:
        if 'aqi' in s.lower():
            sheet = s
            break
    if sheet is None:
        sheet = xls.sheet_names[0]
    df = pd.read_excel(path, sheet_name=sheet)
    df.columns = [str(c).strip() for c in df.columns]
    if 'Day' not in df.columns:
        # try a more tolerant rename: find column name that looks like 'Day'
        for c in df.columns:
            if 'day' in c.lower():
                df = df.rename(columns={c: 'Day'})
                break
    if 'Day' not in df.columns:
        raise ValueError(f"'Day' column missing in {path}; columns: {df.columns.tolist()}")
    month_cols = [c for c in df.columns if c != 'Day']
    long = df.melt(id_vars=['Day'], value_vars=month_cols, var_name='MonthName', value_name='AQI')
    long = long.dropna(subset=['AQI']).reset_index(drop=True)
    # normalize Day to int
    long['Day'] = pd.to_numeric(long['Day'], errors='coerce').astype('Int64')
    # map month name to number
    month_map = {calendar.month_name[i]: i for i in range(1,13)}
    month_map.update({calendar.month_abbr[i]: i for i in range(1,13)})
    long['Month'] = long['MonthName'].map(lambda x: month_map.get(str(x).strip(), np.nan)).astype('Int64')
    # determine year: priority -> year_hint param -> filename -> try to find 4-digit year
    if year_hint is None:
        m = re.search(r"(20\d{2})", os.path.basename(path))
        year_hint = int(m.group(1)) if m else None
    if year_hint is None:
        raise ValueError(f"Cannot determine year for file {path}. Provide year_hint or include year in filename.")
    def make_date(r):
        try:
            return pd.Timestamp(year=year_hint, month=int(r['Month']), day=int(r['Day']))
        except Exception:
            return pd.NaT
    long['Date'] = long.apply(make_date, axis=1)
    long = long.dropna(subset=['Date']).reset_index(drop=True)
    long['AQI'] = pd.to_numeric(long['AQI'], errors='coerce')
    long['Year'] = year_hint
    # Keep only needed cols
    return long[['Date','Year','Month','Day','AQI']]

# Build combined DataFrame
list_parts = []
for f in files:
    try:
        part = melt_calendar(f)
        print(f"Loaded {os.path.basename(f)} -> rows {len(part)}")
        list_parts.append(part)
    except Exception as e:
        print("SKIP file", f, "error:", e)

if not list_parts:
    raise SystemExit("No AQI files could be loaded. Check DATA_ROOT and file format.")

all_long = pd.concat(list_parts, ignore_index=True)
print("Combined rows:", len(all_long))
print(all_long[['Date','Year','Month','Day','AQI']].head())


Found files: []


SystemExit: No AQI files could be loaded. Check DATA_ROOT and file format.

C:\Users\sober\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# AQI_and_Prominent_Plots_notebook.py
# Ready-to-run notebook cells (paste into a .ipynb code cell) that:
# - auto-detects your calendar-style AQI sheets for 2023/2024/2025
# - melts Day x Month -> Date rows
# - reads prominent-pollutant sheets (auto-detected by sheet name)
# - produces and saves all requested PNG charts
# Requirements: pandas, numpy, matplotlib

# ---------------------- CELL 1: Imports & file discovery ----------------------
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import calendar

DATA_DIR = 'D:\Downloads\FieldProjectAQI_Data'
# Pattern-match files you uploaded; adjust if your filenames differ
aqi_files = sorted(glob.glob(os.path.join(DATA_DIR, '*mumbai*202*.xlsx')))
prom_files = sorted(glob.glob(os.path.join(DATA_DIR, 'Prominent*202*.xlsx')) +
                    glob.glob(os.path.join(DATA_DIR, 'prominent*202*.xlsx')) +
                    glob.glob(os.path.join(DATA_DIR, '*prominent*202*.xlsx')))

print('AQI files found:', aqi_files)
print('Prominent pollutant files found:', prom_files)


AQI files found: []
Prominent pollutant files found: []


<>:17: SyntaxWarning: invalid escape sequence '\D'
<>:17: SyntaxWarning: invalid escape sequence '\D'
C:\Users\sober\AppData\Local\Temp\ipykernel_17028\3423529007.py:17: SyntaxWarning: invalid escape sequence '\D'
  DATA_DIR = 'D:\Downloads\FieldProjectAQI_Data'


In [ ]:
# ---------------------- CELL 2: Helpers to melt calendar-style AQI ----------------------

def melt_calendar_aqi(path, year=None):
    """Read a calendar-style sheet (columns: Day, January..December) and return long DataFrame.
    If the file contains multiple sheets, the function chooses the first sheet containing 'AQI' or uses sheet 0.
    """
    xls = pd.ExcelFile(path)
    # choose sheet containing 'AQI' (case-insensitive) else first sheet
    sheet = None
    for s in xls.sheet_names:
        if 'aqi' in s.lower():
            sheet = s
            break
    if sheet is None:
        sheet = xls.sheet_names[0]
    df = pd.read_excel(path, sheet_name=sheet)
    df = df.rename(columns=lambda c: str(c).strip())
    if 'Day' not in df.columns:
        raise ValueError(f"Expected column 'Day' in {path} (sheet: {sheet}). Columns: {df.columns.tolist()}")
    month_cols = [c for c in df.columns if str(c).strip().lower() != 'day']
    long = df.melt(id_vars=['Day'], value_vars=month_cols, var_name='Month', value_name='AQI')
    long = long.dropna(subset=['AQI'])
    long['Day'] = pd.to_numeric(long['Day'], errors='coerce').astype('Int64')
    month_map = {calendar.month_name[i]: i for i in range(1,13)}
    month_map.update({calendar.month_abbr[i]: i for i in range(1,13)})
    long['MonthNum'] = long['Month'].map(lambda x: month_map.get(str(x).strip(), np.nan))
    # If year not provided, try to parse from filename
    if year is None:
        import re
        m = re.search(r"(20\d{2})", os.path.basename(path))
        year = int(m.group(1)) if m else None
    if year is None:
        raise ValueError('Unable to determine year for ' + path)
    def make_date(row):
        try:
            return pd.Timestamp(year=year, month=int(row['MonthNum']), day=int(row['Day']))
        except Exception:
            return pd.NaT
    long['Date'] = long.apply(make_date, axis=1)
    long = long.dropna(subset=['Date']).reset_index(drop=True)
    long['AQI'] = pd.to_numeric(long['AQI'], errors='coerce')
    long['Year'] = year
    long['Month'] = long['MonthNum']
    return long[['Date','Year','Month','Day','AQI']]


In [ ]:
# ---------------------- CELL 3: Load all AQI files into a single DataFrame ----------------------
all_long = []
if not aqi_files:
    raise SystemExit(f'No AQI files found in {DATA_DIR}. Please check the folder and filename pattern.')

for p in aqi_files:
    try:
        df_long = melt_calendar_aqi(p)
        all_long.append(df_long)
    except Exception as e:
        print('Failed to melt', p, '->', str(e))

if not all_long:
    raise SystemExit(f'No AQI long-form data was created. Check file formats and file names in {DATA_DIR}.')

all_long = pd.concat(all_long, ignore_index=True)
all_long['Month'] = all_long['Month'].astype(int)
print('Combined rows:', len(all_long))

# Optional quick preview (uncomment if running interactively)
# display(all_long.head())

SystemExit: No AQI files found in D:\Downloads\FieldProjectAQI_Data. Please check the folder and filename pattern.

C:\Users\sober\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# ---------------------- CELL 4: Read prominent pollutant sheets (auto-detect) ----------------------
# This function looks for a sheet whose name contains 'prominent' or 'dominant' or 'pollutant'

def read_prominent_file(path):
    xls = pd.ExcelFile(path)
    candidate = None
    for s in xls.sheet_names:
        lname = s.lower()
        if any(k in lname for k in ['prominent','dominant','pollutant','param']):
            candidate = s
            break
    if candidate is None:
        candidate = xls.sheet_names[0]
    df = pd.read_excel(path, sheet_name=candidate)
    df = df.rename(columns=lambda c: str(c).strip())
    return df, candidate

prom_long_list = []
for p in prom_files:
    try:
        dfp, sheet = read_prominent_file(p)
        print('Reading', p, 'sheet:', sheet, 'columns:', list(dfp.columns)[:10])
        # Attempt to melt similar calendar-style if it has Day + months
        if 'Day' in dfp.columns:
            months = [c for c in dfp.columns if str(c).strip().lower()!='day']
            tmp = dfp.melt(id_vars=['Day'], value_vars=months, var_name='Month', value_name='Pollutant')
            tmp['Day'] = pd.to_numeric(tmp['Day'], errors='coerce').astype('Int64')
            month_map = {calendar.month_name[i]: i for i in range(1,13)}
            month_map.update({calendar.month_abbr[i]: i for i in range(1,13)})
            tmp['MonthNum'] = tmp['Month'].map(lambda x: month_map.get(str(x).strip(), np.nan))
            # get year from filename
            import re
            m = re.search(r"(20\d{2})", os.path.basename(p))
            year = int(m.group(1)) if m else None
            def make_date(row):
                try:
                    return pd.Timestamp(year=year, month=int(row['MonthNum']), day=int(row['Day']))
                except:
                    return pd.NaT
            tmp['Date'] = tmp.apply(make_date, axis=1)
            tmp = tmp.dropna(subset=['Date']).reset_index(drop=True)
            tmp['Year'] = year
            tmp = tmp[['Date','Year','MonthNum','Day','Pollutant']].rename(columns={'MonthNum':'Month'})
            prom_long_list.append(tmp)
        else:
            print('Prominent sheet not in Day x Month layout; trying to find Date or Day columns.')
            # If it already has a Date column and a Pollutant column
            possible_date = None
            for col in dfp.columns:
                if 'date' in col.lower():
                    possible_date = col
                    break
            poll_col = None
            for col in dfp.columns:
                if any(k in col.lower() for k in ['dominant','prominent','pollutant','param','characteristic']):
                    poll_col = col
                    break
            if possible_date is not None and poll_col is not None:
                tmp = dfp[[possible_date, poll_col]].dropna()
                tmp.columns = ['Date','Pollutant']
                tmp['Date'] = pd.to_datetime(tmp['Date'], errors='coerce')
                tmp = tmp.dropna(subset=['Date']).reset_index(drop=True)
                tmp['Year'] = tmp['Date'].dt.year
                tmp['Month'] = tmp['Date'].dt.month
                prom_long_list.append(tmp[['Date','Year','Month','Pollutant']])
            else:
                print('Could not auto-parse prominent pollutant sheet', p, '- please inspect the sheet manually if parsing fails.')
    except Exception as e:
        print('Error reading prominent file', p, ':', e)

if prom_long_list:
    prom_all = pd.concat(prom_long_list, ignore_index=True)
    prom_all['Pollutant'] = prom_all['Pollutant'].astype(str).str.strip()
    print('Prominent pollutant records:', len(prom_all))
else:
    prom_all = pd.DataFrame()
    print('No prominent pollutant data parsed automatically.')#

No prominent pollutant data parsed automatically.


In [ ]:
# CELL 5 (REVISED): Plotting functions & CPCB categories
# ==============================================================================

def aqi_category(a):
    try:
        a = float(a)
    except (ValueError, TypeError):
        return np.nan
    if a <= 50: return 'Good'
    if a <= 100: return 'Satisfactory'
    if a <= 200: return 'Moderate'
    if a <= 300: return 'Poor'
    if a <= 400: return 'Very Poor'
    return 'Severe'

OUTPUT_DIR = DATA_DIR
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 3-Year Comparative Monthly Averages
monthly_avg = all_long.groupby(['Year','Month'])['AQI'].mean().reset_index()
pivot_monthly = monthly_avg.pivot(index='Month', columns='Year', values='AQI').reindex(index=range(1,13))
plt.figure(figsize=(14, 7)) # Increased figure size for better spacing
pivot_monthly.plot(kind='bar', width=0.8, ax=plt.gca()) # Use ax=plt.gca() to plot on the current figure
plt.xlabel('Month', fontsize=12)
plt.ylabel('Average AQI', fontsize=12)
plt.title('3-Year Comparative Monthly Average AQI (Mumbai)', fontsize=16)
plt.xticks(ticks=range(12), labels=[calendar.month_abbr[i+1] for i in range(12)], rotation=0)
# FIX: Move legend to a better position
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout(rect=[0, 0, 0.9, 1]) # Adjust layout to make space for legend
out1 = os.path.join(OUTPUT_DIR, '3yr_monthly_avg_aqi.png')
plt.savefig(out1)
plt.close()
print('Saved', out1)

# Overall Annual AQI Trend (bar chart for better label visibility)
annual_avg = all_long.groupby('Year')['AQI'].mean().reset_index()
plt.figure(figsize=(7, 5))
# FIX: Switched to a bar chart and adjusted x-ticks for clarity
bars = plt.bar(annual_avg['Year'], annual_avg['AQI'], color=['skyblue', 'salmon', 'lightgreen'])
plt.xlabel('Year', fontsize=12)
plt.ylabel('Average AQI', fontsize=12)
plt.title('Overall Annual Average AQI', fontsize=14)
# FIX: Ensure x-axis ticks are integers for the years
plt.xticks(annual_avg['Year'].astype(int))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
out2 = os.path.join(OUTPUT_DIR, 'annual_avg_aqi.png')
plt.savefig(out2)
plt.close()
print('Saved', out2)

# ... (The code for Daily Trends and Heatmaps remains the same as it didn't have issues) ...
# Daily AQI Trends for Each Year
for y in sorted(all_long['Year'].unique()):
    dfy = all_long[all_long['Year']==y].sort_values('Date')
    plt.figure(figsize=(14,4))
    plt.plot(dfy['Date'], dfy['AQI'])
    plt.xlabel('Date')
    plt.ylabel('AQI')
    plt.title(f'Daily AQI Trend - {y}')
    plt.tight_layout()
    out = os.path.join(OUTPUT_DIR, f'daily_trend_{y}.png')
    plt.savefig(out)
    plt.close()
    print('Saved', out)

# AQI Heatmap (Month vs Day) for each year
for y in sorted(all_long['Year'].unique()):
    dfy = all_long[all_long['Year']==y].copy()
    heat = dfy.pivot_table(index='Month', columns='Day', values='AQI', aggfunc='mean').reindex(index=range(1,13))
    plt.figure(figsize=(14,6))
    plt.imshow(heat, aspect='auto', origin='lower')
    plt.colorbar(label='AQI')
    plt.yticks(ticks=np.arange(12), labels=[calendar.month_name[i+1] for i in range(12)])
    plt.xticks(ticks=np.arange(31), labels=list(range(1,32)))
    plt.xlabel('Day of Month')
    plt.ylabel('Month')
    plt.title(f'AQI Heatmap (Month vs Day) - {y}')
    plt.tight_layout()
    out = os.path.join(OUTPUT_DIR, f'aqi_heatmap_{y}.png')
    plt.savefig(out)
    plt.close()
    print('Saved', out)

# AQI Category Distribution by Year
all_long['Category'] = all_long['AQI'].apply(aqi_category)
cat_counts = all_long.groupby(['Year','Category']).size().reset_index(name='days')
pivot_cat = cat_counts.pivot(index='Category', columns='Year', values='days').reindex(index=['Good','Satisfactory','Moderate','Poor','Very Poor','Severe']).fillna(0)
plt.figure(figsize=(10, 6)) # Increased figure size
pivot_cat.plot(kind='bar', ax=plt.gca())
plt.xlabel('AQI Category', fontsize=12)
plt.ylabel('Days', fontsize=12)
plt.title('AQI Category Distribution by Year', fontsize=16)
# FIX: Rotate labels for better visibility
plt.xticks(rotation=45, ha='right')
# FIX: Move legend to a better position
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout(rect=[0, 0, 0.85, 1]) # Adjust layout for legend
out = os.path.join(OUTPUT_DIR, 'aqi_category_distribution.png')
plt.savefig(out)
plt.close()
print('Saved', out)

# Monthly AQI Distribution Boxplot
monthly_boxes = [all_long[all_long['Month']==m]['AQI'].dropna().values for m in range(1,13)]
plt.figure(figsize=(12,6))
plt.boxplot(monthly_boxes, labels=[calendar.month_abbr[m] for m in range(1,13)])
plt.xlabel('Month', fontsize=12)
plt.ylabel('AQI', fontsize=12)
plt.title('Monthly AQI Distribution (Boxplot) - All years combined', fontsize=16)
plt.tight_layout()
out = os.path.join(OUTPUT_DIR, 'monthly_aqi_boxplot.png')
plt.savefig(out)
plt.close()
print('Saved', out)


KeyError: 'Year'

In [ ]:
# ==============================================================================
# CELL 6 (REVISED): Pollutant-specific charts (if data exists)
# ==============================================================================
if not prom_all.empty:
    prom_all['Year'] = prom_all['Date'].dt.year
    prom_all['Month'] = prom_all['Date'].dt.month
    monthly_prom = prom_all.groupby(['Year','Month','Pollutant']).size().reset_index(name='days')
    for y in sorted(prom_all['Year'].unique()):
        dfy = monthly_prom[monthly_prom['Year']==y]
        pivot = dfy.pivot(index='Month', columns='Pollutant', values='days').reindex(index=range(1,13)).fillna(0)
        plt.figure(figsize=(12, 6)) # Increased figure size
        pivot.plot(kind='bar', stacked=True, ax=plt.gca())
        plt.xlabel('Month', fontsize=12)
        plt.ylabel('Days (count)', fontsize=12)
        plt.title(f'Monthly Prominent Pollutant Breakdown - {y}', fontsize=16)
        plt.xticks(ticks=range(12), labels=[calendar.month_abbr[i+1] for i in range(12)], rotation=0)
        # FIX: Move legend to a better position
        plt.legend(title='Pollutant', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout(rect=[0, 0, 0.8, 1]) # Adjust layout for legend
        out = os.path.join(OUTPUT_DIR, f'monthly_prominent_pollutant_{y}.png')
        plt.savefig(out)
        plt.close()
        print('Saved', out)

    # Overall pollutant dominance (pie)
    overall = prom_all['Pollutant'].value_counts()
    plt.figure(figsize=(10, 8)) # Increased figure size
    # FIX: Create a function to prevent label overlap on the pie chart
    def autopct_generator(limit):
        """Don't show percentage label for slices smaller than limit."""
        def inner_autopct(pct):
            return ('%1.1f%%' % pct) if pct > limit else ''
        return inner_autopct
    
    overall.plot(kind='pie', autopct=autopct_generator(3), ylabel='', textprops={'fontsize': 10})
    plt.title('Overall Pollutant Dominance (All years)', fontsize=16)
    # FIX: Create a legend for the pie chart instead of labels on the slices
    plt.legend(labels=overall.index, bbox_to_anchor=(1.15, 0.9), loc="upper right", title="Pollutants")
    plt.tight_layout()
    out = os.path.join(OUTPUT_DIR, 'overall_pollutant_dominance.png')
    plt.savefig(out)
    plt.close()
    print('Saved', out)
else:
    print('Prominent pollutant dataset is empty — pollutant-specific charts were skipped.')

print('\nAll done. Check the PNG files in', OUTPUT_DIR)


Prominent pollutant dataset is empty — pollutant-specific charts were skipped.

All done. Check the PNG files in D:\Downloads\FieldProjectAQI_Data


In [ ]:
# after running the notebook that produced all_long and prom_all
all_long.to_csv(r'FieldProjectAQI_Data/data/cleaned_aqi.csv', index=False)
if not prom_all.empty:
    prom_all.to_csv(r'FieldProjectAQI_Data/data/prominent_pollutant.csv', index=False)


OSError: Cannot save file into a non-existent directory: 'FieldProjectAQI_Data\data'